# 연습문제 Rsp

A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.

* setA : 	무작위로 가위, 바위, 보를 생성하고, 설정한다. 1000 wei씩 금액을 건다.
* setB	: 외부에서 직접 입력하여 설정한다. 1000 wei씩 금액을 건다.
* play :	가위, 바위, 보의 승패를 정한다.
* distributeBetAmount : 	이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 1000 wei를 가지게 된다.
* getMatchResult()	: "A wins", "B wins", "tie" 결과를 출력

ganache@8345에서 게임을 다음과 같이 진행하세요.

* 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

* A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

* B가 내기금액 걸고, 사용자가 입력한 가위바위보

* 승부내기

* 누가 이겼는지 출력

* 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력

## 1단계 개발

In [4]:
%%writefile src/Rsp.sol
pragma solidity >= 0.6.4;

contract Rsp{
    address owner; //A player
    address addrB; //B player
    uint8 rspA;    //0:rock 1:scissors 2:paper
    uint8 rspB;    //0:rock 1:scissors 2:paper
    int8 result;   //0:tie  1:A wins   2:B wins
    uint betting;
    
    mapping(address=>uint) balanceOf;
    constructor() public{
        owner = msg.sender; //owner account
        betting = 0;        //betting counting
        result = -1;        //strat is mius, this game result
    }
    
    function deposit(uint amount, address _addrB) payable public onlyOwner{
        require(msg.value == amount); //if owner account == amount
        balanceOf[owner] += amount;   //continue plus
        addrB = _addrB;               //Player B's account
        balanceOf[addrB] += amount;
    }
    function getBalanceAll() view public returns(uint, uint, uint){
        return (balanceOf[owner], balanceOf[addrB], address(this).balance);//return all player account
    }
    function setA(uint amount) public onlyOwner{
        require(balanceOf[owner] >= amount);
        rspA = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3); //0 or 1 or 2
        balanceOf[owner] -= amount; //owner's account amount, mius
        betting += amount;          //betting's all money
     }
    function setB(uint8 _rsp, uint amount) public onlyOwner{
        require(balanceOf[addrB] >= amount);
        rspB = _rsp;
        balanceOf[addrB] -= amount;
        betting += amount;
    }
    function play() public{
        if (rspA == rspB){
            result = 0; //tie
        }
        else if ((rspA==0 && rspB==1) || (rspA==1 && rspB==2) || (rspA==2 && rspB==0)){
            result = 1; //A wins
        }
        else{
            result = 2; //B wins
        }
    }
    function distributeBetAmount() payable public{
        if (result == 1){      // A wins
            balanceOf[owner] += betting;
        }
        else if(result == 2){ // B wins
            balanceOf[addrB] += betting;
        }
        else{                  // tie
            balanceOf[owner] += betting/2;
            balanceOf[addrB] += betting/2;
        }
        betting = 0;
    }
    function getMatchResult() view public returns(string memory){
        string memory ret = "";
        
        if(result == 0){
            ret = "tie";
        }
        else if(result == 1){
            ret = "A wins";
        }
        else if(result == 2){
            ret = "B wins";
        }
        return ret;
    }
    modifier onlyOwner{
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/Rsp.sol


## 2단계 컴파일

In [5]:
!solc --abi --bin --gas src/Rsp.sol


======= src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   47469 + 628400 = 675869
external:
   deposit(uint256,address):	65539
   distributeBetAmount():	52170
   getBalanceAll():	3780
   getMatchResult():	infinite
   play():	28418
   setA(uint256):	66616
   setB(uint8,uint256):	66438
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006002819055507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff600160166101000a81548160ff021916908360000b60ff160217905550610c46806100a66000396000f3fe6080604052600436106100705760003560e01c806393e84cd91161004e57806393e84cd91461018c578063a0be28f5146101a3578063cb4bac76146101ad578063ee919d50146101f557610070565b80634a686381146100755780636e553f6514610105578063764e15a414610153575b600080fd5b34801561008157600080fd5b5061008a610230565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100ca5

## 3단계 배포

In [6]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var web3;
if(typeof web3 != 'undefined'){
    web3 = new Web3(web3.currentProvider);
}
else{
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"distributeBetAmount","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalanceAll","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getMatchResult","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"play","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint8","name":"_rsp","type":"uint8"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setB","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "0x"+"608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006002819055507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff600160166101000a81548160ff021916908360000b60ff160217905550610c46806100a66000396000f3fe6080604052600436106100705760003560e01c806393e84cd91161004e57806393e84cd91461018c578063a0be28f5146101a3578063cb4bac76146101ad578063ee919d50146101f557610070565b80634a686381146100755780636e553f6514610105578063764e15a414610153575b600080fd5b34801561008157600080fd5b5061008a610230565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100ca5780820151818401526020810190506100af565b50505050905090810190601f1680156100f75780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6101516004803603604081101561011b57600080fd5b8101908080359060200190929190803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610351565b005b34801561015f57600080fd5b506101686104d8565b60405180848152602001838152602001828152602001935050505060405180910390f35b34801561019857600080fd5b506101a16105ac565b005b6101ab6106ef565b005b3480156101b957600080fd5b506101f3600480360360408110156101d057600080fd5b81019080803560ff1690602001909291908035906020019092919050505061090e565b005b34801561020157600080fd5b5061022e6004803603602081101561021857600080fd5b8101908080359060200190929190505050610a73565b005b6060806040518060200160405280600081525090506000600160169054906101000a900460000b60000b141561029d576040518060400160405280600381526020017f7469650000000000000000000000000000000000000000000000000000000000815250905061034a565b60018060169054906101000a900460000b60000b14156102f4576040518060400160405280600681526020017f412077696e7300000000000000000000000000000000000000000000000000008152509050610349565b6002600160169054906101000a900460000b60000b1415610348576040518060400160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525090505b5b5b8091505090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146103aa57600080fd5b8134146103b657600080fd5b81600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555080600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055508160036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505050565b6000806000600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205447925092509250909192565b600160159054906101000a900460ff1660ff16600160149054906101000a900460ff1660ff1614156105fc576000600160166101000a81548160ff021916908360000b60ff1602179055506106ed565b6000600160149054906101000a900460ff1660ff1614801561062f575060018060159054906101000a900460ff1660ff16145b80610669575060018060149054906101000a900460ff1660ff1614801561066857506002600160159054906101000a900460ff1660ff16145b5b806106a457506002600160149054906101000a900460ff1660ff161480156106a357506000600160159054906101000a900460ff1660ff16145b5b156106cc5760018060166101000a81548160ff021916908360000b60ff1602179055506106ec565b6002600160166101000a81548160ff021916908360000b60ff1602179055505b5b565b60018060169054906101000a900460000b60000b141561077e57600254600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610904565b6002600160169054906101000a900460000b60000b141561080f5760025460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550610903565b600280548161081a57fe5b04600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540192505081905550600280548161089357fe5b0460036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505b5b6000600281905550565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461096757600080fd5b8060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205410156109d557600080fd5b81600160156101000a81548160ff021916908360ff1602179055508060036000600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610acc57600080fd5b80600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020541015610b3957600080fd5b6003424460405160200180838152602001828152602001925050506040516020818303038152906040528051906020012060001c81610b7457fe5b06600160146101000a81548160ff021916908360ff16021790555080600360008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550806002600082825401925050819055505056fea2646970667358221220a5ff869145c3c01915b064cf4156147322bd4ce805da7631028a454dc7efd3fa64736f6c63430006040033";

async function deploy(){
    const accounts = await web3.eth.getAccounts();              //web3's all accounts
    console.log("Deploying the contract from: " + accounts[0]); //fisrt account
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data:_bin})
        .send({from:accounts[0], gas:1000000}, function(err, transactionHash){
            if(!err){
                console.log("hash: "+transactionHash);
            }
        });
    console.log("The contract deployed to: "+deployed.options.address)
}
deploy()

Writing src/RspDeploy.js


In [7]:
!node src/RspDeploy.js

Deploying the contract from: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
hash: 0x174a0f0f49da76455440d8a6dae9104150baf4710ad6ea00b3cc026c745b601c
The contract deployed to: 0x0cAA5152F22c847b4F6366959E5bEB5fdA997215


## 4단계 사용

In [17]:
%%writefile src/RspUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"_addrB","type":"address"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"distributeBetAmount","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalanceAll","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getMatchResult","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"play","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setA","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint8","name":"_rsp","type":"uint8"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"setB","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _rsp = new web3.eth.Contract(_abi, "0x0cAA5152F22c847b4F6366959E5bEB5fdA997215");

async function dolt(){
    const accounts = await web3.eth.getAccounts();  //ganache's all accounts
    
    // player A,B : setting 5000wei
    await _rsp.methods.deposit(5000, accounts[1]).send({from:accounts[0], gas:100000, value:5000});
    
    // Before game : A, B's contract Accounts balanceOf print
    _rsp.methods.getBalanceAll().call().then(function(v) {
        console.log("Before play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2]+"\n")
    });
                                             
    // setA(betting amount) -> 1000
    await _rsp.methods.setA(1000).send({from:accounts[0], gas:100000});
    
    // setB(select rsp && betting amount)
    // rsp(0:rock, 1:scissors, 2:paper)
    await _rsp.methods.setB(1, 1000).send({from:accounts[0], gas:100000});
    
    // winner or loser
    await _rsp.methods.play().send({from:accounts[0], gas:100000});
    
    // print result
    _rsp.methods.getMatchResult().call().then(console.log);
    
    // betting amount distribute
    await _rsp.methods.distributeBetAmount().send({from:accounts[0], gas:100000});
    
    // After game : A,B's contract Accounts balanceOf print
    _rsp.methods.getBalanceAll().call().then(function(v) {
        console.log("\nAfter play\nA: "+v[0]+"  B: "+v[1]+"  contract: "+v[2]+"\n")
    });
}
dolt()

Overwriting src/RspUse.js


* A가 이겼을 때

In [18]:
!node src/RspUse.js

Before play
A: 23000  B: 27000  contract: 25000

A wins

After play
A: 24000  B: 26000  contract: 25000



* B가 이겼을 때

In [19]:
!node src/RspUse.js

Before play
A: 29000  B: 31000  contract: 30000

B wins

After play
A: 28000  B: 32000  contract: 30000



* 비겼을 때

In [20]:
!node src/RspUse.js

Before play
A: 33000  B: 37000  contract: 35000

tie

After play
A: 33000  B: 37000  contract: 35000

